## 1. Initialize Your Environment

In [47]:
## RUN THIS CELL BEFORE PROCEEDING; LATER CODE BLOCKS RELY ON THIS
%run -i 'scripts/isaac_magic.py'
%isaac --reset --status

Isaac call will open new connection
Logfile: Ready!
Servers: Ready!
IP Addr: 3.87.169.70


In [123]:
%%isaac
import asyncio
from omni.isaac.core.utils.stage import add_reference_to_stage, get_current_stage
from omni.isaac.core.utils.nucleus import get_assets_root_path
from omni.physx.scripts import physicsUtils  

from omni.isaac.core.materials.physics_material import PhysicsMaterial
from omni.isaac.core.prims.geometry_prim import GeometryPrim

from omni.isaac.examples.base_sample import BaseSample
from pxr import UsdGeom, Gf, UsdPhysics

from omni.isaac.franka.controllers import PickPlaceController
from omni.isaac.franka import Franka

import numpy as np
import omni

class HelloManipulator(BaseSample):
    def __init__(self) -> None:
        super().__init__()

        self.Can_URL = "/dli/task/Can.usdz" # 캔 usd
        self.Table_URL = "/dli/task/Scifi_Table_Concept.usdz" # 투명 테이블 usd
        self.Basket_URL = "/dli/task/Basket.usdz" # 바구니 usd
        self.Beer_URL = "/dli/task/Beer_Bottle.usdz" # 맥주병 usd
        
        # 프랑카 로봇이 여러번 집기 및 놓기 작업을 위한 위치 리스트
        self.pick_place_steps = [
            {"picking_position": np.array([0.29, -0.19, 0.18]), "placing_position": np.array([0.3, 0.3, 0.45])},
            {"picking_position": np.array([0.3, -0.52, 0.12]), "placing_position": np.array([0.33, -0.54, 0.19])},
            {"picking_position": np.array([0.30, -0.55, 0.17]), "placing_position": np.array([0.30, -0.55, 0.17])},# 0.3 0.3 0.45
            {"picking_position": np.array([0.23, -0.4479, 0.175]), "placing_position": np.array([0.29, -0.19, 0.25])},
            {"picking_position": np.array([0.38, -0.18, 0.18]), "placing_position": np.array([0.32, 0.3, 0.45])}
        ]
        self.current_step = 0 # 프랑카 로봇의 작업 현재 단계 인덱스
        return
    
    # 로봇 소환 - 프랑카 로봇 인스턴스를 생성하여 시뮬레이션 월드에 추가
    def add_robot(self):
        self._franka = self._world.scene.add(
            Franka(
                prim_path="/World/Franka",
                name="Franka",
            )
        )
        return

    def add_objs(self):
        
        ################# can ################
        
        self._can = add_reference_to_stage(usd_path=self.Can_URL, prim_path=f"/World/Can")
        self._stage = omni.usd.get_context().get_stage()

        # modify can mesh
        can_mesh = UsdGeom.Mesh.Get(self._stage, "/World/Can")
        physicsUtils.set_or_add_translate_op(can_mesh, translate=Gf.Vec3f(0.3, -0.3, 0.5))
        physicsUtils.set_or_add_scale_op(can_mesh, scale=Gf.Vec3f(0.0003, 0.0003, 0.0003))
        physicsUtils.set_or_add_orient_op(can_mesh, orient=Gf.Quatf(0.9238795, 0.3826834, 0, 0))

        # add rigid body        
        can_rigid = UsdPhysics.RigidBodyAPI.Apply(get_current_stage().GetPrimAtPath("/World/Can"))
        # add collision
        can_geom = GeometryPrim(prim_path="/World/Can", name="can_ref_geom", collision=True)
        can_geom.set_collision_approximation("convexDecomposition")
        
        ################ Table ###############
        
        self._table = add_reference_to_stage(usd_path=self.Table_URL, prim_path=f"/World/Table")
        self._stage = omni.usd.get_context().get_stage()

        # modify Table mesh
        table_mesh = UsdGeom.Mesh.Get(self._stage, "/World/Table")
        physicsUtils.set_or_add_translate_op(table_mesh, translate=Gf.Vec3f(0.3, -0.3, 0.2))
        physicsUtils.set_or_add_scale_op(table_mesh, scale=Gf.Vec3f(0.0015, 0.0015, 0.0016))
        physicsUtils.set_or_add_orient_op(table_mesh, orient=Gf.Quatf(0.7071068,0.7071068, 0, 0))

        # add rigid body        
        table_rigid = UsdPhysics.RigidBodyAPI.Apply(get_current_stage().GetPrimAtPath("/World/Table"))
        # add collision
        table_geom = GeometryPrim(prim_path="/World/Table", name="table_ref_geom", collision=True)
        table_geom.set_collision_approximation("convexDecomposition")
        
        #################### Basket ########################
        
        self._basket = add_reference_to_stage(usd_path=self.Basket_URL, prim_path=f"/World/Basket")
        self._stage = omni.usd.get_context().get_stage()

        # modify Basket mesh
        basket_mesh = UsdGeom.Mesh.Get(self._stage, "/World/Basket")
        physicsUtils.set_or_add_translate_op(basket_mesh, translate=Gf.Vec3f(0.3, 0.3, 0.20))
        physicsUtils.set_or_add_scale_op(basket_mesh, scale=Gf.Vec3f(0.0015, 0.0015, 0.0015))
        physicsUtils.set_or_add_orient_op(basket_mesh, orient=Gf.Quatf(0.7071068,0.7071068, 0, 0))

        # add rigid body        
        basket_rigid = UsdPhysics.RigidBodyAPI.Apply(get_current_stage().GetPrimAtPath("/World/Basket"))
        # add collision
        basket_geom = GeometryPrim(prim_path="/World/Basket", name="basket_ref_geom", collision=True)
        basket_geom.set_collision_approximation("convexDecomposition")
        
        ######################### Beer Bottle ######################
        
        self._beer = add_reference_to_stage(usd_path=self.Beer_URL, prim_path=f"/World/Beer")
        self._stage = omni.usd.get_context().get_stage()

        # modify Beer Bottle mesh
        beer_mesh = UsdGeom.Mesh.Get(self._stage, "/World/Beer")
        physicsUtils.set_or_add_translate_op(beer_mesh, translate=Gf.Vec3f(0.3, -0.55, 0.5))
        physicsUtils.set_or_add_scale_op(beer_mesh, scale=Gf.Vec3f(0.0006, 0.0006, 0.0006))
        physicsUtils.set_or_add_orient_op(beer_mesh, orient=Gf.Quatf(0.9238795, 0.3826834, 0, 0))
        
        # add rigid body        
        beer_rigid = UsdPhysics.RigidBodyAPI.Apply(get_current_stage().GetPrimAtPath("/World/Beer"))
        # add collision
        beer_geom = GeometryPrim(prim_path="/World/Beer", name="beer_ref_geom", collision=True)
        beer_geom.set_collision_approximation("convexDecomposition")
        # beer_mesh.CreateMassAttr().Set(3.0) # 해당 부분으로 무게 세팅을 하면 함수 충돌이 일어나므로  아래 코드로 무게 세팅 가능.
        # 병의 무게가 달라질 경우 프랑카 로봇의 집기 놓기 작업 위치가 달라져야함.        
        # 병의 질량 설정
        # mass_api = UsdPhysics.MassAPI.Apply(get_current_stage().GetPrimAtPath("/World/Beer"))
        # mass_api.GetMassAttr().Set(0.5)  # 질량을 0.5으로 설정 
        
        
        return

    # 시뮬레이션 환경 설정 - 월드에 기본 지면 추가 후, 로봇과 오브젝트들을 초기화
    def setup_scene(self):
        world = self.get_world()
        world.scene.add_default_ground_plane()

        self.add_robot()
        self.add_objs()
        return

    # 시뮬레이션에서 사용할 물리적 특성 설정
    async def setup_materials(self):
        self.high_friction_material = PhysicsMaterial(
            prim_path="/World/Physics_Materials/high_friction_material",
            name="high_friction_material",
            static_friction=1.0,
            dynamic_friction=1.0,
        )
        await self._world.reset_async()
        return
    
    async def atach_materials(self):
        geometry_prim = GeometryPrim(prim_path="/World/Can")
        geometry_prim.apply_physics_material(self.high_friction_material)
        await self._world.play_async()
        return

    # 시뮬레이션 시작 후 초기화
    async def setup_post_load(self):
        self._world = self.get_world()

        await self.setup_materials()
        await self.atach_materials()

        self._controller = PickPlaceController(
            name="pick_place_controller",
            gripper_dof_indices = self._franka.gripper.dof_indices,
            robot_articulation=self._franka,
        )

        self._articulation_controller = self._franka.get_articulation_controller()
        self._world.add_physics_callback("sim_step", callback_fn=self.physics_step)
        return

    # 프랑카 로봇의 집기 및 놓기 동작 수행
    def physics_step(self, step_size):
        
        joints_state = self._franka.get_joints_state()
        
        # 현재 단계의 집기 및 놓기 위치를 가져옴
        current_pick_place_step = self.pick_place_steps[self.current_step]
        
        actions = self._controller.forward(
            picking_position=current_pick_place_step["picking_position"],
            placing_position=current_pick_place_step["placing_position"],
            current_joint_positions=joints_state.positions,
        )
        
        # 각 하나 하나 마다 단계를 돌음
        if self._controller.is_done():
            self.current_step += 1 # step 갱신
            if self.current_step < len(self.pick_place_steps):
                # 다음 단계로 넘어가며 컨트롤러 리셋
                self._controller.reset()
            else:
                # 모든 단계가 완료되면 시뮬레이션 일시 정지
                self._world.pause()
        else:
            self._articulation_controller.apply_action(actions)
        return

Passing cell content via `telnet localhost 8223`
DONE


In [124]:
%%isaac
def _on_load_world():
    async def _on_load_world_async():
        await HelloManipulator().load_world_async()
    asyncio.ensure_future(_on_load_world_async())

_on_load_world()

Passing cell content via `telnet localhost 8223`
DONE
